In [1]:
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns
sns.set_theme()

import torch 
import torch.nn as nn 
import torch.optim as optim


from models.learned_qp_solver_bi_cycle_steer_part import MLP_Init, learned_qp_solver
from scipy.io import loadmat
from torch.utils.data import Dataset, DataLoader
# torch.set_default_dtype(torch.float64)

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using {device} device")

Using cuda device


In [3]:


num = 50
t = 0.1


steer_min = -0.8
steer_max = 0.8

steerdot_min = -1.0
steerdot_max = 1.0

steerddot_min = -1.0
steerddot_max = 1.0



In [4]:
#### creating dataset 

dataset_size = 200000

steer_samples = np.random.uniform(steer_min, steer_max,  (dataset_size, num)    )
# vel_samples = np.random.multivariate_normal(np.zeros(num), vel_max*np.identity(num), (dataset_size, ) )



steer_init = np.random.uniform(steer_min, steer_max, (dataset_size, 1)  )


inp = np.hstack(( steer_samples,  steer_init ))

median_ = np.median(inp, axis=0)
q1 = np.percentile(inp, 25, axis=0)
q3 = np.percentile(inp, 75, axis=0)
iqr_ = q3 - q1
# Handle constant features
iqr_ = np.where(iqr_ == 0, 1, iqr_)

print(np.array_repr(median_))
print(np.array_repr(iqr_))


median_ = torch.tensor(median_).float().to(device)
iqr_ = torch.tensor(iqr_).float().to(device)




array([-4.30842576e-04,  5.91636819e-05, -1.98533905e-03,  1.95326047e-03,
       -1.39358004e-03, -5.68464714e-05,  2.65373538e-04, -2.05526464e-03,
       -7.79943244e-04, -2.89536127e-04, -1.67782600e-03, -1.30369985e-03,
        1.54351594e-03, -2.10914668e-03, -8.94449061e-04, -1.12731106e-04,
        1.39067091e-03,  2.35456389e-03,  1.13105755e-03, -1.60172509e-03,
        5.96616154e-04,  4.50916455e-04,  1.03440312e-03,  2.20521562e-03,
        1.91066337e-03, -2.36909941e-03,  5.23557830e-04,  3.22519684e-03,
       -6.10364108e-04, -1.55881571e-03, -2.54650519e-04,  5.49939732e-04,
       -2.54372567e-03,  1.15624681e-03,  3.09751082e-03, -1.01707749e-03,
       -1.16847934e-03, -9.60229520e-04,  3.19715942e-04, -7.80995571e-04,
        3.91363172e-04,  1.96635947e-05, -2.01634184e-04, -1.34935042e-03,
       -1.25220145e-03,  1.28156314e-04,  1.63297072e-03, -2.31602556e-03,
        8.75998110e-04, -2.06147938e-04,  1.15305881e-03])
array([0.80001846, 0.79929592, 0.79676276

In [5]:
# Custom Dataset Loader 
class TrajDataset(Dataset):
	"""Expert Trajectory Dataset."""
	def __init__(self, inp, steer_init, steer_samples):
		
		# State Data
		self.inp = inp
  
		self.steer_init = steer_init
		
		# Expert Coeff
		self.steer_samples = steer_samples

		
		
	
	def __len__(self):
		return len(self.inp)    
			
	def __getitem__(self, idx):
		
		# Inputs
		inp = self.inp[idx]
		steer_init = self.steer_init[idx]

		
		# Outputs
		steer_samples = self.steer_samples[idx]
	
		
		
		return torch.tensor(inp).float(), torch.tensor(steer_init).float(), torch.tensor(steer_samples).float()
	

# Batch Size - 3k or 4k
batch_size = 2056

# pcd_data = pcd_data.reshape(data_set_size, 2, 200)

# Using PyTorch Dataloader
train_dataset = TrajDataset(inp, steer_init,  steer_samples)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0, drop_last=True)



In [6]:
# Differentiable Layer
num_batch = train_loader.batch_size


mlp_init_inp_dim = np.shape(inp)[1]
mlp_init_hidden_dim = 1024
# mlp_init_out_dim = (2*nvar)**2+2*nvar+2*nvar
mlp_init_out_dim = num+(2*num+2*(num-1)+2*(num-2))
# print((2*num+2*(num-1)+2*(num-2)))

# mlp_pred = MLP_Pred(mlp_pred_inp_dim, mlp_pred_hidden_dim, mlp_pred_out_dim  )
mlp_init = MLP_Init(mlp_init_inp_dim, mlp_init_hidden_dim, mlp_init_out_dim  )


model = learned_qp_solver(num_batch, num, t, mlp_init).to(device)
# model.load_state_dict(torch.load('./weights/learned_qp_bi_cycle_vel_part.pth'))



In [7]:
epochs = 800
step = 0 
beta = 1.0 # 3.5

optimizer = optim.AdamW(model.parameters(), lr = 1e-4, weight_decay=6e-5)
# optimizer = optim.RMSprop(model.parameters(), lr = 1e-3, weight_decay=6e-5)

# scheduler = optim.lr_scheduler.StepLR(optimizer, step_size = 100, gamma = 0.1)

avg_train_loss, avg_loss_primal, avg_fixed_point_loss, avg_proj_loss = [], [], [], []
model.train()
for epoch in range(epochs):
	
	# Train Loop
	losses_train, aug_losses_primal, aug_losses_fixed_point, aug_losses_proj = [], [], [], []
	
	for (inp, steer_init, steer_samples) in train_loader:
		
		# Input and Output 
  
		################################################################################################

		# print(goal_des[0])
		# print(y_lb[0], y_ub[0])

  
		inp = inp.to(device)
		steer_init = steer_init.to(device)
		
		steer_samples = steer_samples.to(device)
	
		
		steer_projected, res_primal_stack, res_fixed_point_stack, accumulated_res_primal, accumulated_res_fixed_point = model(inp, steer_init, steer_samples, steer_max, steer_min, steerdot_max, steerdot_min, steerddot_max, steerddot_min, median_, iqr_)
		loss, primal_loss, fixed_point_loss, proj_loss = model.ss_loss(accumulated_res_primal, accumulated_res_fixed_point, steer_projected, steer_samples)
	
		optimizer.zero_grad()
		loss.backward()
		# torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=0.5)
		optimizer.step()
		
		losses_train.append(loss.detach().cpu().numpy()) 
		aug_losses_primal.append(primal_loss.detach().cpu().numpy())
		aug_losses_fixed_point.append(fixed_point_loss.detach().cpu().numpy())
		aug_losses_proj.append(proj_loss.detach().cpu().numpy())
		# aug_losses_steer.append(steer_loss.detach().cpu().numpy())

		
	# scale = scale*1.2	
		
	if epoch % 4 == 0:    
		print(f"Epoch: {epoch + 1}, Train Loss: {np.average(losses_train):.3f}, aug_loss_primal: {np.average(aug_losses_primal):.3f}, aug_loss_fixed_point: {np.average(aug_losses_fixed_point):.3f}, aug_loss_proj: {np.average(aug_losses_proj):.3f} ")

	step += 0.15 #0.15
	# scheduler.step()
	avg_train_loss.append(np.average(losses_train)), avg_loss_primal.append(np.average(aug_losses_primal)), avg_fixed_point_loss.append(np.average(aug_losses_fixed_point)),  avg_proj_loss.append(np.average(aug_losses_proj))
	
		



Epoch: 1, Train Loss: 0.518, aug_loss_primal: 0.000, aug_loss_fixed_point: 0.402, aug_loss_proj: 0.116 
Epoch: 5, Train Loss: 0.159, aug_loss_primal: 0.000, aug_loss_fixed_point: 0.048, aug_loss_proj: 0.111 
Epoch: 9, Train Loss: 0.142, aug_loss_primal: 0.000, aug_loss_fixed_point: 0.033, aug_loss_proj: 0.109 
Epoch: 13, Train Loss: 0.135, aug_loss_primal: 0.000, aug_loss_fixed_point: 0.027, aug_loss_proj: 0.109 
Epoch: 17, Train Loss: 0.130, aug_loss_primal: 0.000, aug_loss_fixed_point: 0.022, aug_loss_proj: 0.108 
Epoch: 21, Train Loss: 0.127, aug_loss_primal: 0.000, aug_loss_fixed_point: 0.019, aug_loss_proj: 0.108 
Epoch: 25, Train Loss: 0.125, aug_loss_primal: 0.000, aug_loss_fixed_point: 0.017, aug_loss_proj: 0.107 
Epoch: 29, Train Loss: 0.123, aug_loss_primal: 0.000, aug_loss_fixed_point: 0.016, aug_loss_proj: 0.107 
Epoch: 33, Train Loss: 0.121, aug_loss_primal: 0.000, aug_loss_fixed_point: 0.014, aug_loss_proj: 0.107 
Epoch: 37, Train Loss: 0.120, aug_loss_primal: 0.000, aug_

KeyboardInterrupt: 

In [8]:
torch.save(model.state_dict(), './weights/learned_qp_bi_cycle_steer_part.pth')